### Problem Definition

### Classical code

In [1]:
import json

In [2]:
conf = json.load(open("conf_24.json","r"))

In [4]:
N = conf["N"] # Number of Teachers
M = conf["M"] # Number of Classes
H = conf["H"] # Number of hours in week

random_seed = conf["random_seed"]
shots = conf["shots"] * 5
layers = conf["layers"]

bin_length = (M * N * H) + (M * H) + (N * H)

print("Requried Qubits: ", bin_length)

Requried Qubits:  24


In [5]:
# T_i is binary string denoting the available hours for each i Teacher
# C_j is binary string denoting the available hours for each j Class

T = conf["T"]
C = conf["C"]

R = conf["R"] # R_ij must be either 0 or 1

equations = []

#### Binary string generator 

In [6]:
def binary_string(x, bin_length):
    emp = ""
    for i in range(bin_length, 0, -1):
        emp += str(x % 2)
        x //= 2
    return emp

#### Objective

In [7]:
def f(i, j, h):
  return int(T[i-1][h-1] == C[j-1][h-1] == '1')

def tau_value(bit_string, j, h):
  return int(bit_string[(h-1) * M + (j - 1)])

def lambda_value(bit_string, i, h):
  return int(bit_string[(H * M) + ((h-1) * N + (i - 1))])

def f_value(bit_string, i, j, h):

  return int(bit_string[(H * M) + (N * H) + ((i - 1) * H * M) + ((j - 1) * H) + (h - 1)])

In [8]:
def objective(bit_string):
  cost = 0
  bit_string = bit_string[::-1]

  # equation a
  for i in range(1, N + 1):
    for j in range(1, M + 1):
      for h in range(1, H + 1):
        if f(i, j, h) != f_value(bit_string, i, j, h):
          cost += 1
            
  # equation b
  for i in range(1, N + 1):
    for j in range(1, M + 1):
      temp = 0
      for h in range(1, H + 1):
        temp += f_value(bit_string, i, j, h)
      cost += (temp - R[i-1][j-1])**2

  # equation c
  for j in range(1, M + 1):
    for h in range(1, H + 1):
      temp = 0
      for i in range(1, N + 1):
        temp += f_value(bit_string, i, j, h)
      cost += (temp + tau_value(bit_string, j, h) - 1)**2

  # equation d
  for i in range(1, N + 1):
    for h in range(1, H + 1):
      temp = 0
      for j in range(1, M + 1):
        temp += f_value(bit_string, i, j, h)
      cost += (temp + lambda_value(bit_string, i, h) - 1)**2

  return cost

#### Solution

In [9]:
def solve(N, M, H):
    sol = bin_length
    sol_str = ""
    for i in range(2 ** bin_length):
        bin_str = binary_string(i, bin_length)
        ob = objective(bin_str)
        
        if(sol > ob):
          print(ob , bin_str)
          sol = ob
          sol_str = bin_str
          
          if (sol == 0):
             break

In [10]:
solve(N , M , H)

16 000000000000000000000000
12 100000000000000000000000
10 100000000100000000000000
8 100000000001000000000000
7 100000000001010000000000
6 100000000001011000000000
5 100000000001011100000000
4 100000000001011110000000
3 100000000001011110010000
2 100000000001011110011000
1 100000000001011110011100
0 100000000001011110011110
